In [2]:
import requests
from requests.auth import HTTPBasicAuth
from flask import Flask, request
import threading
import webbrowser
import time
import pandas as pd

# Variáveis Globais
app = Flask(__name__)
authorization_code = None

# Configurações OAuth
client_id = '419759cc09659588aa42c22986968016e4ce2adc'
client_secret = 'ab1097471db5ec489f4e285b35098d95bb27c4469eb6abeabb99fecf46ea'
redirect_url = 'http://localhost:5000/callback'
state = "8bde85dd6e729dcd6e0d01dde003469d"
authorization_url = f"https://www.bling.com.br/Api/v3/oauth/authorize?response_type=code&client_id={client_id}&redirect_uri={redirect_url}&state={state}"
token_url = "https://www.bling.com.br/Api/v3/oauth/token"
max_pages = 5
page_limit = 500
id_deposito_escritorio = 863558208


@app.route('/callback')
def callback():
    global authorization_code
    authorization_code = request.args.get('code')
    return "Authorization code recebido. Você pode fechar esta janela."

def start_flask_app():
    app.run(port=5000)

def get_authorization_code(client_id, redirect_uri):
    threading.Thread(target=start_flask_app).start()
    time.sleep(1)
    state = "8bde85dd6e729dcd6e0d01dde003469d"
    url = f"https://www.bling.com.br/Api/v3/oauth/authorize?response_type=code&client_id={client_id}&redirect_uri={redirect_uri}&state={state}"
    webbrowser.open(url)
    # Aguarda até que o `authorization_code` seja definido
    while not authorization_code:
        time.sleep(1)
  
    return authorization_code

def get_access_token(client_id, client_secret, authorization_code):
    url = "https://www.bling.com.br/Api/v3/oauth/token"
    headers = {
        "Content-Type": "application/x-www-form-urlencoded",
        "Accept": "1.0"
    }
    body = {
        "grant_type": "authorization_code",
        "code": authorization_code,
        "redirect_uri": "http://localhost:5000/callback"
    }
    response = requests.post(url, headers=headers, data=body, auth=HTTPBasicAuth(client_id, client_secret))
    return response.json()


# Obter o authorization_code
authorization_code = get_authorization_code(CLIENT_ID, REDIRECT_URI)

# Obter o access_token usando o authorization_code
token_response = get_access_token(CLIENT_ID, CLIENT_SECRET, authorization_code)
access_token = token_response.get('access_token')

# CONSULTAR TODAS AS PAGINAS DOS PRODUTOS
max_pag = 5
pagina = 1
df_final = []

headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
}

# Loop para percorrer várias páginas
for _ in range(max_pag):
    url = f"https://www.bling.com.br/Api/v3/produtos?pagina={pagina}&limite=500&criterio=1&tipo=T"
    response = requests.get(url, headers=headers)
    data = response.json()
    display(data)
    # Verifica se há produtos na página
    produtos = data.get('data', [])
    if not produtos:
        break

    # Criando um DataFrame com os produtos
    prod_df = pd.DataFrame(produtos)
    df_final.append(prod_df)

    # Incrementa a página para a próxima iteração
    pagina += 1

# Combina todos os DataFrames em um único DataFrame
df_produtos = pd.concat(df_final, ignore_index=True)

# Mostra as primeiras linhas do DataFrame combinado
display(df_produtos)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


KeyboardInterrupt: 

127.0.0.1 - - [21/May/2024 17:13:39] "GET /callback?code=4a7f90799cb559ecbb90edcc255dd4434f53e827&state=8bde85dd6e729dcd6e0d01dde003469d HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2024 17:14:34] "GET /callback?code=95ccf713e33aa5e726deb01516509776944e2721&state=8bde85dd6e729dcd6e0d01dde003469d HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2024 17:18:16] "GET /callback?code=4596653645093e322161d8c58cb11de9b01af381&state=8bde85dd6e729dcd6e0d01dde003469d HTTP/1.1" 200 -


In [26]:
while True:
    url = f'https://www.bling.com.br/Api/v3/produtos?pagina=217&limite=500&criterio=2&tipo=T'
    resposta = requests.get(url, headers=headers)
    dados = resposta.json()
    if len(dados['data']) < 1:
        break

display(dados)

In [12]:
for i, header in enumerate(df_combined):
    print(df_combined['id'][i], df_combined['codigo'][i])

16238622505 295753
16238622499 295752
16238622495 295751
16238622488 295750
16238622480 295749
16238622469 295748
16238611972 295747
16238611954 295746
16238611933 295745
16238611913 295744


In [13]:
def consultar_estoque():
    # Configuração da API
    base_url = "https://www.bling.com.br/Api/v3/estoques/saldos"
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }

    # Solicitação de dados do usuário
    for i, header in enumerate(df_combined):
        id_produto = df_combined['id'][i]
        codigo_produto = df_combined['codigo'][i]

        # Montagem da URL com parâmetros
        url = f"{base_url}?idsProdutos[]={id_produto}&codigo={codigo_produto}"

        # Fazendo a requisição à API
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()

        # Extraindo os dados de saldo
        saldo_info = data.get('data', [])
        if saldo_info:
            produto_info = saldo_info[0].get('produto', {})
            saldo_fisico_total = saldo_info[0].get('saldoFisicoTotal', 'N/A')
            saldo_virtual_total = saldo_info[0].get('saldoVirtualTotal', 'N/A')
            depositos_info = saldo_info[0].get('depositos', [])

            # Criando um DataFrame com as informações do saldo
            df = pd.DataFrame(depositos_info)
            df['produto_id'] = produto_info.get('id', 'N/A')
            df['produto_codigo'] = codigo_produto
            df['saldo_fisico_total'] = saldo_fisico_total
            df['saldo_virtual_total'] = saldo_virtual_total
            df.rename(columns={
                'id': 'deposito_id',
                'saldoFisico': 'saldo_fisico',
                'saldoVirtual': 'saldo_virtual'
            }, inplace=True)

            display(df)
        else:
                print("Nenhum saldo encontrado para o produto.")
    else:
        print("Erro ao acessar a API:", response.status_code, response.text)

# Chamando a função para testar
consultar_estoque()


,deposito_id,saldo_fisico,saldo_virtual,produto_id,produto_codigo,saldo_fisico_total,saldo_virtual_total
0,863558208,1,1,16238622505,295753,1,1
1,6938657703,0,0,16238622505,295753,1,1


,deposito_id,saldo_fisico,saldo_virtual,produto_id,produto_codigo,saldo_fisico_total,saldo_virtual_total
0,863558208,1,1,16238622499,295752,1,1
1,6938657703,0,0,16238622499,295752,1,1


,deposito_id,saldo_fisico,saldo_virtual,produto_id,produto_codigo,saldo_fisico_total,saldo_virtual_total
0,863558208,1,1,16238622495,295751,1,1
1,6938657703,0,0,16238622495,295751,1,1


,deposito_id,saldo_fisico,saldo_virtual,produto_id,produto_codigo,saldo_fisico_total,saldo_virtual_total
0,863558208,1,1,16238622488,295750,1,1
1,6938657703,0,0,16238622488,295750,1,1


,deposito_id,saldo_fisico,saldo_virtual,produto_id,produto_codigo,saldo_fisico_total,saldo_virtual_total
0,863558208,1,1,16238622480,295749,1,1
1,6938657703,0,0,16238622480,295749,1,1


,deposito_id,saldo_fisico,saldo_virtual,produto_id,produto_codigo,saldo_fisico_total,saldo_virtual_total
0,863558208,0,0,16238622469,295748,0,0
1,6938657703,0,0,16238622469,295748,0,0


,deposito_id,saldo_fisico,saldo_virtual,produto_id,produto_codigo,saldo_fisico_total,saldo_virtual_total
0,863558208,1,1,16238611972,295747,1,1
1,6938657703,0,0,16238611972,295747,1,1


,deposito_id,saldo_fisico,saldo_virtual,produto_id,produto_codigo,saldo_fisico_total,saldo_virtual_total
0,863558208,2,2,16238611954,295746,2,2
1,6938657703,0,0,16238611954,295746,2,2


,deposito_id,saldo_fisico,saldo_virtual,produto_id,produto_codigo,saldo_fisico_total,saldo_virtual_total
0,863558208,0,0,16238611933,295745,0,0
1,6938657703,0,0,16238611933,295745,0,0


,deposito_id,saldo_fisico,saldo_virtual,produto_id,produto_codigo,saldo_fisico_total,saldo_virtual_total
0,863558208,0,0,16238611913,295744,0,0
1,6938657703,0,0,16238611913,295744,0,0


In [21]:
url = "https://www.bling.com.br/Api/v3/depositos?pagina=1&limite=100&situacao=1"

headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
}

response = requests.get(url, headers=headers)
data = response.json()
#display(data)# Vamos assumir que a chave para acessar os produtos é 'data' na resposta JSON
depositos = data['data']

# Criando um DataFrame com os produtos
df = pd.DataFrame(depositos)
display(df)

,id,descricao,situacao,padrao,desconsiderarSaldo
0,863558208,Escritório,1,True,False
1,6938657703,Loja física,1,False,False


In [143]:
import requests
from requests.auth import HTTPBasicAuth
from flask import Flask, request
import threading
import webbrowser
import time
import pandas as pd

# Variáveis Globais
app = Flask(__name__)
authorization_code = None

# Configurações OAuth
CLIENT_ID = '419759cc09659588aa42c22986968016e4ce2adc'
CLIENT_SECRET = 'ab1097471db5ec489f4e285b35098d95bb27c4469eb6abeabb99fecf46ea'
REDIRECT_URI = 'http://localhost:5000/callback'
STATE = "8bde85dd6e729dcd6e0d01dde003469d"
AUTHORIZATION_URL = f"https://www.bling.com.br/Api/v3/oauth/authorize?response_type=code&client_id={CLIENT_ID}&redirect_uri={REDIRECT_URI}&state={STATE}"
TOKEN_URL = "https://www.bling.com.br/Api/v3/oauth/token"
ID_DEPOSITO_ESCRITORIO = 863558208
PAGE_LIMIT = 500
MAX_PAGES = int(input('Quantas paginas consultaremos? '))  # Limite de páginas para teste

@app.route('/callback')
def callback():
    global authorization_code
    authorization_code = request.args.get('code')
    return "Authorization code recebido. Você pode fechar esta janela."

def start_flask_app():
    app.run(port=5000)

def get_authorization_code(client_id, redirect_uri):
    threading.Thread(target=start_flask_app).start()
    time.sleep(1)
    
    url = f"https://www.bling.com.br/Api/v3/oauth/authorize?response_type=code&client_id={client_id}&redirect_uri={redirect_uri}&state={STATE}"
    webbrowser.open(url)

    # Aguarda até que o `authorization_code` seja definido
    while not authorization_code:
        time.sleep(1)
    
    return authorization_code

def get_access_token(client_id, client_secret, authorization_code):
    url = "https://www.bling.com.br/Api/v3/oauth/token"
    headers = {
        "Content-Type": "application/x-www-form-urlencoded",
        "Accept": "1.0"
    }
    body = {
        "grant_type": "authorization_code",
        "code": authorization_code,
        "redirect_uri": "http://localhost:5000/callback"
    }
    response = requests.post(url, headers=headers, data=body, auth=HTTPBasicAuth(client_id, client_secret))
    return response.json()

def fetch_all_products(access_token, page_limit, max_pages=None):
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }
    pagina = 1
    df_final = []

    while True:
        url = f"https://www.bling.com.br/Api/v3/produtos?pagina={pagina}&limite={page_limit}&criterio=1&tipo=T"
        response = requests.get(url, headers=headers)
        data = response.json()

        produtos = data.get('data', [])
        if not produtos or (max_pages and pagina > max_pages):
            break

        prod_df = pd.DataFrame(produtos)
        df_final.append(prod_df)

        pagina += 1

    return pd.concat(df_final, ignore_index=True), headers

# Função para consultar os saldos de estoque de múltiplos produtos
def consultar_saldos_produtos(prod_ids, headers):
    base_url = "https://www.bling.com.br/Api/v3/estoques/saldos"
    params = [('idsProdutos[]', prod_id) for prod_id in prod_ids]

    response = requests.get(base_url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json().get('data', [])
        saldos = []
        for item in data:
            produto_info = item.get('produto', {})
            saldo_escritorio = next(
                (d['saldoVirtual'] for d in item.get('depositos', []) if d['id'] == ID_DEPOSITO_ESCRITORIO),
                0  # Default para 0
            )
            saldos.append({
                'produto_id': produto_info.get('id', 'N/A'),
                'saldo_escritorio': saldo_escritorio
            })
        return saldos
    else:
        print(f"Erro ao acessar a API: {response.status_code}, {response.text}")
        return []

# Função para obter os saldos e adicionar ao DataFrame, filtrando apenas produtos com estoque > 0
def obter_estoque_produtos(prod_df, headers):
    batch_size = 20  # Tamanho do lote para cada requisição de saldo
    saldos = []

    for start in range(0, len(prod_df), batch_size):
        end = min(start + batch_size, len(prod_df))
        batch_df = prod_df.iloc[start:end]
        batch_ids = batch_df['id'].tolist()
        saldos += consultar_saldos_produtos(batch_ids, headers)

    saldos_df = pd.DataFrame(saldos)

    # Mesclar o DataFrame de produtos com os saldos de estoque
    merged_df = prod_df.merge(saldos_df, left_on='id', right_on='produto_id', how='left').drop(columns=['produto_id'])
    
    # Filtrar apenas produtos com saldo maior que zero
    return merged_df[merged_df['saldo_escritorio'] > 0]

# Main
if __name__ == "__main__":
    # Obtenha o authorization_code
    authorization_code = get_authorization_code(CLIENT_ID, REDIRECT_URI)
    token_response = get_access_token(CLIENT_ID, CLIENT_SECRET, authorization_code)
    access_token = token_response.get('access_token')

    # Obtenha todos os produtos
    df_combined, headers = fetch_all_products(access_token, PAGE_LIMIT, MAX_PAGES)

    # Adiciona a coluna de estoque ao DataFrame e filtra produtos com estoque > 0
    df_combined = obter_estoque_produtos(df_combined, headers)

    # Mostra as primeiras linhas do DataFrame combinado
    display(df_combined)

    # Exibe apenas as colunas 'id' e 'codigo'
    display(df_combined[['id', 'codigo']])


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [142]:
df_combined.to_excel("produtos.xlsx", index=False)